In [1]:
!pip install pytorch-lightning segmentation_models_pytorch tacoreader rasterio albumentations --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from datamodule import CoreDataModule
from model import UNet_CD_Sentinel_2
import config
import metrics
import tacoreader
import torch

In [3]:
# Primeiro Treinamento com as imagens 512x512
tb_logger = TensorBoardLogger(config.DIR_LOG, name=config.NAME_MOBILENET)

dataset_l1c = tacoreader.load(["/content/drive/MyDrive/taco_CloudSen12/cloudsen12-l2a.0000.part.taco",
                              "/content/drive/MyDrive/taco_CloudSen12/cloudsen12-l2a.0001.part.taco",
                              "/content/drive/MyDrive/taco_CloudSen12/cloudsen12-l2a.0002.part.taco",
                              "/content/drive/MyDrive/taco_CloudSen12/cloudsen12-l2a.0003.part.taco",
                              "/content/drive/MyDrive/taco_CloudSen12/cloudsen12-l2a.0004.part.taco",
                              "/content/drive/MyDrive/taco_CloudSen12/cloudsen12-l2a.0005.part.taco",
                              ])
df_512 = dataset_l1c[(dataset_l1c["label_type"] == "high") & (dataset_l1c["real_proj_shape"] == 509)]

# Define the datamodule
datamodule = CoreDataModule(
    dataframe=df_512,
    batch_size=config.BATCH_SIZE
)

# Define the model
model = UNet_CD_Sentinel_2(
    encoder_name=config.ENCODER_NAME_MOBILENET,
    classes=config.CLASSES,
    in_channels=config.IN_CHANNELS,
    learning_rate=config.LEARNING_RATE,
)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=config.DIR_ROOT_MOBILENET,
    filename="{epoch}-{train_loss:.2f}-{val_loss:.2f}-trainHigh512",
    monitor="val_loss",
    mode="min",
    save_top_k=1,
)

earlystopping_callback = pl.callbacks.EarlyStopping(
    monitor="val_loss", patience=12, mode="min"
)

callbacks = [checkpoint_callback, earlystopping_callback]

# Define the trainer
trainer = pl.Trainer(
    max_epochs=config.EPOCHS,
    log_every_n_steps=1,
    callbacks=callbacks,
    accelerator=config.ACCELERATOR,
    precision="16-mixed",
    logger=tb_logger,
    default_root_dir=config.DIR_ROOT_MOBILENET,
)


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [4]:
torch.set_float32_matmul_precision('medium')
# Start the training
trainer.fit(model=model, datamodule=datamodule)

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /content/drive/MyDrive/Unet_4_bands_l2a/lightning_logs/Unet_mobilenet_v2 exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params | Mode 
---------------------------------------------------
0 | model | Unet             | 6.6 M  | train
1 | loss  | CrossEntropyLoss | 0      | train
---------------------------------------------------
6.6 M     Trainable params
0         Non-trainable params
6.6 M     Total params
26.519    Total estimated model params size (MB)
285       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [5]:
# Carregar o melhor modelo diretamente
model = UNet_CD_Sentinel_2.load_from_checkpoint(
    checkpoint_callback.best_model_path,
    encoder_name=config.ENCODER_NAME_MOBILENET,
    classes=config.CLASSES,
    in_channels=config.IN_CHANNELS,
    learning_rate=config.LEARNING_RATE,
)

# run val dataset
val_metrics = trainer.validate(model, datamodule=datamodule, verbose=True)
print(val_metrics)

# run test dataset
test_metrics = trainer.test(model, datamodule=datamodule, verbose=True)
print(test_metrics)

acuracia, iou, f1_score, recall, precision, sensitivity = metrics.calculate_metrics(datamodule.test_dataloader(), model.model)

# Salva o modelo treinado
smp_model = model.model
smp_model.save_pretrained(config.DIR_ROOT_MOBILENET + "/" + config.NAME_MOBILENET)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    0.28340399265289307    │
│      valid_acuracia       │     0.945473313331604     │
│     valid_dataset_iou     │    0.8033397197723389     │
│      valid_f1_score       │    0.8909466862678528     │
└───────────────────────────┴───────────────────────────┘

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[{'val_loss': 0.28340399265289307, 'valid_acuracia': 0.945473313331604, 'valid_dataset_iou': 0.8033397197723389, 'valid_f1_score': 0.8909466862678528}]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_acuracia       │     0.946064293384552     │
│     test_dataset_iou      │    0.8052635192871094     │
│       test_f1_score       │    0.8921285271644592     │
│         test_loss         │    0.28506746888160706    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.28506746888160706, 'test_acuracia': 0.946064293384552, 'test_dataset_iou': 0.8052635192871094, 'test_f1_score': 0.8921285271644592}]


Calculando métricas: 100%|██████████| 31/31 [05:08<00:00,  9.96s/it]


Redução: micro
Acurácia no conjunto de teste: 0.9461
IoU no conjunto de teste: 0.8053
F1 no conjunto de teste: 0.8921
Recall no conjunto de teste: 0.8921
Precisião no conjunto de teste: 0.8921
Sensibilidade no conjunto de teste: 0.8921


Redução: macro
Acurácia no conjunto de teste: 0.9461
IoU no conjunto de teste: 0.7093
F1 no conjunto de teste: 0.8196
Recall no conjunto de teste: 0.8314
Precisião no conjunto de teste: 0.8095
Sensibilidade no conjunto de teste: 0.8314


Redução: micro-imagewise
Acurácia no conjunto de teste: 0.9461
IoU no conjunto de teste: 0.8245
F1 no conjunto de teste: 0.8921
Recall no conjunto de teste: 0.8921
Precisião no conjunto de teste: 0.8921
Sensibilidade no conjunto de teste: 0.8921


Redução: macro-imagewise
Acurácia no conjunto de teste: 0.9461
IoU no conjunto de teste: 0.7244
F1 no conjunto de teste: 0.7830
Recall no conjunto de teste: 0.8331
Precisião no conjunto de teste: 0.8474
Sensibilidade no conjunto de teste: 0.8331


